In [60]:
import pandas as pd
import numpy as np

from os import listdir
from os.path import join

In [61]:
data_type = "0506"
base_dir = f"./{data_type}/"

In [62]:
insertions = {
    0: ["Intercept", ""],
    1: ["Age", ""],
    2: ["Diabete", ""],
    5: ["BMI", ""],
    6: ["Smoke", ""],
    8: ["Country", ""],
    10: ["Race", ""],
    14: ["Education", ""],
    19: ["Gender", ""],
    21: ["Activity", ""]
}

In [63]:
columns = ["Variables", "Confidence Interval"]

In [66]:
for file_name in listdir(base_dir):
    assert file_name.endswith("csv")
    df = pd.read_csv(join(base_dir, file_name))
    data = []
    for i in range(df.shape[0]):
        if i in insertions.keys():
            data.append(insertions[i])
        line = df.iloc[i]
        name = line["Unnamed: 0"]
        estm = round(float(line["Estimate"]), 3)
        lb = round(float(line['LB']), 3)
        ub = round(float(line['UB']), 3)
        dataline = [name, f"{estm} ({lb}~{ub})",]
        data.append(dataline)
    form = pd.DataFrame(data, index=None, columns=columns)
    form.to_csv(f"./Result_{data_type}/{file_name}", index=False)

In [65]:
form

,Variables,Confidence Interval
0,Intercept,
1,(Intercept),-0.033 (-0.299~0.232)
2,Age,
3,age,0.001 (0.001~0.001)
4,Diabete,
5,diabPre-diabete,0.021 (-0.24~0.282)
6,diabdiabete,0.001 (-0.263~0.264)
7,diabUnknown,0.005 (-0.256~0.265)
8,BMI,
9,bmi,-0.0 (-0.001~0.0)


In [16]:
df.columns

Index(['Unnamed: 0', 'Estimate', 'Std. Error', 't value', 'Pr(>|t|)', 'LB',
       'UB'],
      dtype='object')